In [ ]:
#pip install transformers trl accelerate torch bitsandbytes peft datasets -qU

In [1]:
from datasets_lit import read_pretrained_qa_file
import json

qa_data = read_pretrained_qa_file('../fine-tune/data-user/extract-qa2.md')
jsonl_file = "../fine-tune/results/qa.jsonl"
with open(jsonl_file, 'w', encoding='utf-8') as f:
   for qa in qa_data:
      data = {
         'instruction': qa['Instruction'],
         'input': qa['Question'],
         'output': qa['Answer'],
         'history': []
      }
      row = json.dumps(data)
      f.write(row + "\r\n")
   f.flush()
   

In [ ]:
from datasets_lit import load_jsonl_dataset
instruct_tune_dataset = load_jsonl_dataset(jsonl_file)

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16,
)

MODEL_PATH = f"../models/Mistral-7B-Instruct-v0.1"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=nf4_config,
    device_map='cuda',
    local_files_only=True,
    #trust_remote_code=False,
    use_cache=False
)

In [5]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [34]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj','up_proj','down_proj','lm_head']
)

In [35]:
base_model = prepare_model_for_kbit_training(model)
peft_model = get_peft_model(base_model, peft_config)

In [56]:
from transformers import TrainingArguments

args = TrainingArguments(
  output_dir = "./results/mistral_instruct_generation",
  #num_train_epochs=5,
  max_steps = 5,
  per_device_train_batch_size=1,
  gradient_accumulation_steps=32,
  warmup_steps = 0.03,
  logging_steps = 10,
  save_strategy="epoch",
  #evaluation_strategy="epoch",
  #evaluation_strategy="steps",
  #eval_steps=20, # comment out this line if you want to evaluate at the end of each epoch
  learning_rate=1e-4,
  bf16=True,
  lr_scheduler_type='constant',
  #report_to="tensorboard"
)

In [ ]:
def create_prompt(sample):
  bos_token = "<s>"
  eos_token = "</s>"
  full_prompt = bos_token
  full_prompt += "### Instruction:\n"
  full_prompt += sample['instruction']
  full_prompt += "\n\n### Input:\n"
  full_prompt += sample["input"]
  full_prompt += "\n\n### Response:\n"
  full_prompt += sample["output"]
  full_prompt += eos_token
  return full_prompt

In [57]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
  model=model,
  peft_config=peft_config,
  max_seq_length=max_seq_length,
  tokenizer=tokenizer,
  packing=True,
  formatting_func=create_prompt, # this will aplly the create_prompt mapping to all training and test dataset
  args=args,
  train_dataset=instruct_tune_dataset["train"],
  #eval_dataset=instruct_tune_dataset["test"]
)

[codecarbon INFO @ 21:17:48] [setup] RAM Tracking...
[codecarbon INFO @ 21:17:48] [setup] GPU Tracking...
[codecarbon INFO @ 21:17:48] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:17:48] [setup] CPU Tracking...
[codecarbon WARNING @ 21:17:48] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon WARNING @ 21:17:50] We saw that you have a 12th Gen Intel(R) Core(TM) i7-12700K but we don't know it. Please contact us.
[codecarbon INFO @ 21:17:50] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-12700K
[codecarbon INFO @ 21:17:50] >>> Tracker's metadata:
[codecarbon INFO @ 21:17:50]   Platform system: Linux-5.15.133.1-microsoft-standard-WSL2-x86_64-with-glibc2.35
[codecarbon INFO @ 21:17:50]   Python version: 3.10.12
[codecarbon INFO @ 21:17:50]   CodeCarbon version: 2.2.3
[codecarbon INFO @ 21:17:50]   Available RAM : 15.619 GB
[codecarbon INFO @ 21:17:50]   CPU count: 20
[codecarbon INFO @ 21:17:50]   CPU model: 12th Gen Intel(R) Core(TM) 

In [60]:
trainer.train(resume_from_checkpoint=True)

[codecarbon WARNING @ 22:49:21] Already started tracking
/home/flash/miniconda3/envs/torch/lib/python3.10/site-packages/trl/trainer/utils.py:570: UserWarning: The dataset reached end and the iterator is reset to the start.
  warnings.warn("The dataset reached end and the iterator is reset to the start.")


Step,Training Loss,Validation Loss


[codecarbon WARNING @ 22:53:00] Tracker already stopped !


TrainOutput(global_step=5, training_loss=0.28147599697113035, metrics={'train_runtime': 219.1465, 'train_samples_per_second': 0.018, 'train_steps_per_second': 0.009, 'total_flos': 894657626112000.0, 'train_loss': 0.28147599697113035, 'epoch': 0.02})

In [59]:
trainer.save_model("./results/mistral_instruct_generation")